In [ ]:
import numpy as np
import pandas as pd
import utils
import catboost

## part 1

In [ ]:
full_train = pd.read_csv("data/train_part_1_v2.csv", dtype=utils.DTYPES, index_col='id')

In [ ]:
closest_hits_features = pd.read_csv('more_features_fixed_part_1.csv',dtype=np.float32)

In [ ]:
train_concat = pd.concat(
    [full_train.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

## part 2

In [ ]:
full_train2 = pd.read_csv("data/train_part_2_v2.csv", dtype=utils.DTYPES, index_col='id')

In [ ]:
closest_hits_features2 = pd.read_csv('more_features_fixed_part_2.csv',dtype=np.float32)

In [ ]:
#fix indexes
a = closest_hits_features2.index.tolist()
a = list(np.array(a) + 2722852)
closest_hits_features2.index = a

In [ ]:
train_concat2 = pd.concat(
    [full_train2.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features2], axis=1)

## concat train data

In [ ]:
train_concat = pd.concat([train_concat, train_concat2], axis=0)

In [ ]:
train_concat.shape

## pretrain work

In [ ]:
labels = pd.concat([full_train.label, full_train2.label], axis=0)

In [ ]:
weights = pd.concat([full_train.weight, full_train2.weight],axis=0)

In [ ]:
def wrapper(value):
    if value < 0:
        value = 0
    return value
abs_weights = weights.apply(wrapper)

## train catboost model

In [ ]:
model = catboost.CatBoostClassifier(loss_function='Logloss',
                                    l2_leaf_reg=20,
                                    border_count=200,
                                    max_depth=6,
                                    bootstrap_type='No', 
                                    random_strength=0, 
                                    iterations=1600, 
                                    learning_rate=0.05, 
                                    verbose=False,
                                    thread_count=4, 
                                    task_type='GPU', 
                                    random_state=12,
                                    rsm=1,
)


In [ ]:
model.fit(train_concat, labels, sample_weight=abs_weights, plot=True)

In [ ]:
model.save_model('track_2_baseline/track_2_model.cbm')